In [97]:
# Colab-only: mount Drive to keep the cursor and CSV between sessions.
# If you don't want Drive, set USE_DRIVE=False below.
USE_DRIVE = True
if USE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

BASE_DIR = '/content/drive/MyDrive/jira_pagination_demo' if USE_DRIVE else '/content'
!mkdir -p "$BASE_DIR"
print("Using base dir:", BASE_DIR)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Using base dir: /content/drive/MyDrive/jira_pagination_demo


In [98]:
# @title Setup
!pip -q install requests python-dateutil

import os, json, time, csv, math, sys
import requests
from datetime import datetime, timezone
from dateutil import parser as dtparse
import pandas as pd

JIRA_BASE_URL = "https://testsite.atlassian.net"  # @param {type:"string"}
JIRA_EMAIL = "myname@gmail.com"                        # @param {type:"string"}
JIRA_API_TOKEN = "xzxzxzxzxzxzxzxzxzxzxzxzx-zzxzxzxzxzxzx-xcxcxcxcxcxcx-cscscscsccsc"               # @param {type:"string"}

AUTH = (JIRA_EMAIL, JIRA_API_TOKEN)
SEARCH_URL_GET = f"{JIRA_BASE_URL}/rest/api/3/search/jql"    # Enhanced Search (GET)
SEARCH_URL_POST = f"{JIRA_BASE_URL}/rest/api/3/search/jql"   # Enhanced Search (POST identical path)
DEFAULT_FIELDS = ["summary","status","assignee","created","updated","issuetype","project","priority"]


# Your query: narrow by project and time if you like; ordering is critical
JQL = """
project = GUM
ORDER BY updated ASC, id ASC
""".strip()

FIELDS = [
    "summary","status","assignee","updated","created","labels","issuetype","priority","reporter","project"
]
MAX_RESULTS = 200  # Jira typically caps at 100 or 1000; 200 is safe. Adjust for your site.

HEADERS = {"Accept": "application/json", "Content-Type": "application/json"}

# ---- Paths ----
CURSOR_PATH = os.path.join(BASE_DIR, "jira_cursor.json")
CSV_PATH    = os.path.join(BASE_DIR, "jira_issues.csv")

# ---- Behavior toggles ----
USE_CURSOR_PAGINATION = True   # If your Jira supports nextPageToken (Enhanced Search)
FALLBACK_OFFSET       = True   # Fallback to startAt if no nextPageToken is returned
SLEEP_SECONDS         = 0.8    # gentle pacing

print("Cursor path:", CURSOR_PATH)
print("CSV path:", CSV_PATH)



Cursor path: /content/drive/MyDrive/jira_pagination_demo/jira_cursor.json
CSV path: /content/drive/MyDrive/jira_pagination_demo/jira_issues.csv


In [99]:
def load_cursor():
    if not os.path.exists(CURSOR_PATH):
        # Initialize a conservative cursor.
        # last_updated: very old date to fetch everything once; you can seed it to a recent checkpoint.
        # last_issue_id: 0 so any real id is > 0
        return {
            "last_updated": "1970-01-01T00:00:00.000+0000",
            "last_issue_id": "0",
            "nextPageToken": None,
            "done": False,
        }
    with open(CURSOR_PATH, "r") as f:
        return json.load(f)

def save_cursor(cur):
    tmp = CURSOR_PATH + ".tmp"
    with open(tmp, "w") as f:
        json.dump(cur, f, indent=2)
    os.replace(tmp, CURSOR_PATH)

def parse_jira_time(ts):
    # Handles both ...+0000 and Z forms.
    try:
        return dtparse.parse(ts)
    except Exception:
        return dtparse.parse(ts.replace("Z", "+00:00"))

def issue_sort_key(issue):
    # Stable sort key: (updated, id)
    updated = issue["fields"].get("updated") or "1970-01-01T00:00:00.000+0000"
    return (parse_jira_time(updated), int(issue["id"]))


In [100]:
def jira_search(enhanced_token=None, start_at=None):
    """
    Enhanced search (cursor-based).
    Falls back to offset only if /search/jql is unavailable (very unlikely on Cloud).
    """
    # Prefer the new enhanced search endpoint (cursor-based)
    url = f"{JIRA_BASE_URL}/rest/api/3/search/jql"
    payload = {"jql": JQL, "fields": FIELDS, "maxResults": MAX_RESULTS}

    # Cursor page (preferred)
    if USE_CURSOR_PAGINATION and enhanced_token:
        payload["nextPageToken"] = enhanced_token

    # If you *really* must use offset pagination (legacy behavior), you can still
    # pass startAt via query for some transitional setups. Avoid unless necessary.
    if (not enhanced_token) and (start_at is not None) and FALLBACK_OFFSET:
        url = f"{url}?startAt={start_at}&maxResults={MAX_RESULTS}"

    resp = requests.post(url, headers=HEADERS, auth=(JIRA_EMAIL, JIRA_API_TOKEN), json=payload)
    if resp.status_code == 429:
        retry_after = int(resp.headers.get("Retry-After", "5"))
        time.sleep(retry_after)
        return jira_search(enhanced_token=enhanced_token, start_at=start_at)
    resp.raise_for_status()
    data = resp.json()

    # Enhanced search returns nextPageToken/isLast and no total count.
    next_token = data.get("nextPageToken") or data.get("nextPage") or None
    issues = data.get("issues", [])
    is_last = data.get("isLast", next_token is None)

    # For transitional environments, you might still see startAt/total. Normalize:
    start_at_resp = data.get("startAt")
    total = data.get("total")
    max_results = data.get("maxResults", MAX_RESULTS)

    return {
        "issues": issues,
        "nextPageToken": next_token,
        "startAt": start_at_resp,
        "total": total,
        "maxResults": max_results,
        "isLast": is_last
    }


In [101]:
from collections import OrderedDict

def ensure_csv_header(path):
    exists = os.path.exists(path)
    cols = ["key","id","updated","created","summary","status","assignee","labels","reporter","project","issue_type","priority","url"]
    if not exists:
        with open(path, "w", newline="", encoding="utf-8") as f:
            writer = csv.DictWriter(f, fieldnames=cols)
            writer.writeheader()
    return cols

def row_from_issue(issue):
    fields = issue["fields"]
    key = issue["key"]
    url = f"{JIRA_BASE_URL}/browse/{key}"
    return OrderedDict({
        "key": key,
        "id": issue["id"],
        "updated": fields.get("updated"),
        "created": fields.get("created"),
        "summary": fields.get("summary"),
        "status": (fields.get("status") or {}).get("name") if fields.get("status") else None,
        "assignee": ((fields.get("assignee") or {}).get("emailAddress") if fields.get("assignee") else None),
        "labels": ",".join(fields.get("labels") or []),
        "reporter": ((fields.get("reporter") or {}).get("emailAddress") if fields.get("reporter") else None),
        "project": ((fields.get("project") or {}).get("key") if fields.get("project") else None),
        "issue_type": ((fields.get("issuetype") or {}).get("name") if fields.get("issuetype") else None),
        "priority": ((fields.get("priority") or {}).get("name") if fields.get("priority") else None),
        "url": url,
    })



In [102]:
cursor = load_cursor()
cols = ensure_csv_header(CSV_PATH)

seen = set()  # cheap local dedupe during this run
start_at = 0  # offset fallback

while True:
    # Prefer cursor token if we have one
    result = jira_search(enhanced_token=cursor.get("nextPageToken"), start_at=(None if cursor.get("nextPageToken") else start_at if FALLBACK_OFFSET else None))
    issues = result["issues"]

    if not issues:
        # No more data
        cursor["nextPageToken"] = result.get("nextPageToken")
        cursor["done"] = True
        save_cursor(cursor)
        print("Done. No issues in this page.")
        break

    # Sort (defensive) and apply boundary filter to enforce idempotence
    issues.sort(key=issue_sort_key)
    boundary_updated = parse_jira_time(cursor["last_updated"])
    boundary_id = int(cursor["last_issue_id"])

    filtered = []
    for it in issues:
        upd = parse_jira_time(it["fields"]["updated"])
        iid = int(it["id"])
        # Skip anything strictly behind the cursor
        if (upd < boundary_updated) or (upd == boundary_updated and iid <= boundary_id):
            continue
        filtered.append(it)

    if not filtered and result.get("isLast"):
        # We're at the end and nothing new beyond boundary
        cursor["nextPageToken"] = result.get("nextPageToken")
        cursor["done"] = True
        save_cursor(cursor)
        print("Reached end with nothing beyond boundary.")
        break

    # Write filtered rows, dedup locally
    with open(CSV_PATH, "a", newline="", encoding="utf-8") as f:
        writer = csv.DictWriter(f, fieldnames=cols)
        for it in filtered:
            rid = it["id"]
            if rid in seen:
                continue
            row = row_from_issue(it)
            writer.writerow(row)
            seen.add(rid)

            # Move cursor forward for each successfully written row
            cursor["last_updated"] = it["fields"]["updated"]
            cursor["last_issue_id"] = it["id"]

    # Persist cursor & page token
    cursor["nextPageToken"] = result.get("nextPageToken")
    save_cursor(cursor)

    # Advance offset if using fallback pagination
    if (not cursor.get("nextPageToken")) and FALLBACK_OFFSET:
        start_at = (result.get("startAt") or 0) + len(issues)

    print(f"Wrote {len(filtered)} rows, cursor now at {cursor['last_updated']} / {cursor['last_issue_id']}")
    time.sleep(SLEEP_SECONDS)

print("All done. Output CSV:", CSV_PATH)


Wrote 3 rows, cursor now at 2025-10-02T09:09:17.297+0000 / 15240
Reached end with nothing beyond boundary.
All done. Output CSV: /content/drive/MyDrive/jira_pagination_demo/jira_issues.csv


In [103]:
df = pd.read_csv(CSV_PATH)
print("Rows:", len(df))
df.head(10)


Rows: 3


,key,id,updated,created,summary,status,assignee,labels,reporter,project,issue_type,priority,url
0,GUM-1,15238,2025-10-02T09:09:09.974+0000,2025-10-02T09:09:09.887+0000,The test issue,Backlog,NaN,NaN,hlosukwakha@gmail.com,GUM,Story,Medium,https://hlosukwakha.atlassian.net/browse/GUM-1
1,GUM-2,15239,2025-10-02T09:09:10.101+0000,2025-10-02T09:09:09.986+0000,The second test issue,Backlog,NaN,NaN,hlosukwakha@gmail.com,GUM,Story,Medium,https://hlosukwakha.atlassian.net/browse/GUM-2
2,GUM-3,15240,2025-10-02T09:09:17.297+0000,2025-10-02T09:09:17.191+0000,The third test issue,Backlog,NaN,NaN,hlosukwakha@gmail.com,GUM,Story,Medium,https://hlosukwakha.atlassian.net/browse/GUM-3
